In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from PIL import Image
import quartz
import matplotlib.pyplot as plt
import sinabs.layers as sl
import numpy as np

np.set_printoptions(suppress=True)

In [ ]:
torch.hub._validate_not_a_forked_repo = lambda a, b, c: True
model = torch.hub.load("pytorch/vision:v0.10.0", "vgg13", pretrained=True)
model.eval();

In [ ]:
import urllib

url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try:
    urllib.URLopener().retrieve(url, filename)
except:
    urllib.request.urlretrieve(url, filename)

In [ ]:
input_image = Image.open(filename)
preprocess = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to("cuda")
    model.to("cuda")

with torch.no_grad():
    output = model(input_batch)

# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)

In [ ]:
# Read the categories
with open("../imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

In [ ]:
batch_size = 1
t_max = 256

# flatten model structure
ann = nn.Sequential(*model.features, *model.classifier)
snn = quartz.from_torch.from_model(ann, t_max=t_max, batch_size=batch_size)

In [ ]:
snn

In [ ]:
ann

In [ ]:
torchvision.datasets.ImageNet("../data", split="val")

In [ ]:
test = nn.MaxPool2d(2)